In [2]:
pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 8.1 MB/s eta 0:00:00


In [3]:
import os
import anthropic
import re
from difflib import SequenceMatcher

# Set up Claude API Key
client = anthropic.Anthropic(api_key="")

def generate_product_sense_question():
    """
    Automatically generates a Product Sense question while incorporating the CIRCLE framework.

    AI dynamically selects:
    - The product domain (General, Industry-Specific, or Trend-Based)
    - The type of product sense question (Problem-Solving, User-Centric, Strategy, or Innovation)
    - Ensures candidates apply the CIRCLE framework in their response.

    :return: (str) AI-generated Product Sense interview question.
    """

    # AI-driven dynamic question prompt with CIRCLE framework
    prompt = """
    You are an interviewer in a product manager job interview. Generate a **single, brief, high-quality Product Sense interview question**
    that feels natural and human-like. The question must encourage candidates to apply the **CIRCLE framework** in their response.

    **Definition of Product Sense:**
    A case study that tests the interviewee’s ability to design a product with minimal information, demonstrating
    critical thinking and communication skills.

    **CIRCLE Framework:**
    The question should require the candidate to:
    1.**Clarify the Goal** – Identify the problem space and define success metrics.
    2.**Identify Customer Personas** – Define the key users and their needs.
    3.**Report Customer Needs** – Articulate pain points and key requirements.
    4.**Conduct Competitive Analysis** – Assess existing solutions and market landscape.
    5.**List Solutions** – Generate possible product solutions.
    6.**Evaluate Trade-offs & Recommend** – Prioritize and justify the best solution.

    **Rules for the Question:**
    - AI should **automatically select** a relevant question type:
        - **Problem-Solving:** "How would you improve X?"
        - **User-Centric:** "How would you design a product for Y persona?"
        - **Strategy-Oriented:** "How would you prioritize Z features?"
        - **Innovation-Based:** "Create a new product for X market."
    - The question must be **open-ended** to allow structured thinking.

    **Examples:**
    - Improve AirBnB – How would you enhance the booking experience for international travelers?
    - Design a mental wellness app for healthcare workers – How would you help frontline medical staff manage stress?
    - Redesign airport security – How would you improve TSA check-in for a smoother experience?
    - Create a competitor for IMDb – How would you build a next-gen movie discovery platform?

    **Generate only ONE brief and well-structured Product Sense interview question that requires the candidate to apply CIRCLE:**
    """

    try:
        response = client.messages.create(
            model="claude-3-7-sonnet-20250219",
            max_tokens=100,
            system="You generate only one structured and high-quality Product Sense interview question, ensuring the candidate applies the CIRCLE framework.",
            messages=[{"role": "user", "content": prompt}],
        )

        return response.content[0].text.strip()

    except Exception as e:
        print(f"⚠️ API Error: {e}")  #  Debugging output
        return f"Error: {str(e)}"


# Stores previous Q&A for each session
conversation_history = {}

# Quality thresholds
QUALITY_THRESHOLD = 0.6  # Minimum similarity score for relevance
MIN_QUALITY_SCORE = 6  # Minimum score for an AI-generated question


def feedback_info_final(session_id):
    """ Generates structured feedback on the entire conversation based on the modified CIRCLE framework. """
    if session_id not in conversation_history or len(conversation_history[session_id]) == 0:
        raise ValueError("No conversation history found for this session.")

    # Extract the entire conversation history
    conversation = conversation_history[session_id]
    formatted_conversation = "\n\n".join(
        [f"**Q:** {entry['question']}\n**A:** {entry['answer']}" for entry in conversation]
    )

    prompt = f"""
    Evaluate the candidate's product sense interview performance using the entire conversation history.

    **Conversation History:**
    {formatted_conversation}

    **Evaluation Criteria (CIRCLE Framework):**
    - **Comprehend the Question**: Did the candidate ask clarifying questions and understand the mission?
    - **Identify the Customer**: Did they recognize a two-sided marketplace (e.g., consumers and producers)?
    - **Cut/Prioritization (Customer Segmentation)**: Did they prioritize a specific customer segment with clear reasoning?
    - **Report Customer Needs/Pain Points**: Did they outline at least three key pain points?
    - **Cut/Prioritization (Pain Points)**: Did they prioritize one pain point and justify it?
    - **List Solutions**: Did they propose at least three solutions, including a moonshot idea?
    - **Cut/Prioritization (Solutions)**: Did they prioritize one solution and explain why?
    - **Estimate Trade-Offs**: Did they analyze trade-offs between solutions?
    - **Summarize**: Did they provide a concise, structured summary?

    **Additional Evaluation Metrics:**
    - **Clarity and Conciseness**: Was the response structured and free from unnecessary details?
    - **Relevance**: Did the response directly address the question and showcase skills?
    - **Problem-Solving**: Did the candidate demonstrate effective challenge resolution?
    - **Customer Focus**: Did the response show a deep understanding of user needs?

    **Final Output Format:**
    - **Evaluation Summary**
    - **What Went Well** (4-5 strengths)
    - **What Can Be Improved** (3-4 areas for improvement)
    - **Next Steps** (Actionable improvement suggestions)
    """

    try:
        response = client.messages.create(
            model="claude-3-7-sonnet-20250219",
            max_tokens=2048,
            system="Generate structured interview feedback using the modified CIRCLE framework for the entire conversation.",
            messages=[{"role": "user", "content": prompt}],
        )

        return response.content[0].text.strip()

    except Exception as e:
        print(f"Error generating final feedback: {e}")
        return None


def quality_control_check(question_text, answer_text, ai_response):
    """ Ensures AI-generated follow-up questions meet quality standards. """
    try:
        # Step 1: Contextual Relevance Check
        relevance_prompt = f"""
        Analyze the following follow-up question for contextual relevance.

        Candidate's Response: {answer_text}
        AI-Generated Follow-Up Question: {ai_response}

        On a scale from **1 to 10**, how well does this follow-up question logically relate to the candidate's response?
        (A score of 1 means it is completely unrelated, 10 means it is perfectly relevant.)

        Score (just a number, no explanation):
        """

        validation_response = client.messages.create(
            model="claude-3-7-sonnet-20250219",
            max_tokens=100,
            system="You verify AI responses for relevance.",
            messages=[{"role": "user", "content": relevance_prompt}],
        )

        score_text = validation_response.content[0].text.strip()
        score_match = re.search(r"\d+", score_text)
        relevance_score = int(score_match.group()) if score_match else 5  # Default to 5 if extraction fails

        if relevance_score < 6:
            print("Warning: AI-generated question may not be contextually relevant!")
            return {"valid": False, "reason": f"Low relevance score ({relevance_score}/10)."}

        # Step 2: Scoring System for Response Quality
        scoring_prompt = f"""
        Evaluate the quality of the AI-generated follow-up question based on clarity, relevance, and depth.
        Give a score from 1 to 10, with **ONLY the number** as the response.

        Question: {question_text}
        Candidate Response: {answer_text}
        AI-Generated Follow-Up Question: {ai_response}

        Score (just a number, no explanation):
        """

        scoring_response = client.messages.create(
            model="claude-3-7-sonnet-20250219",
            max_tokens=100,
            system="You score AI responses for clarity and quality.",
            messages=[{"role": "user", "content": scoring_prompt}],
        )

        score_text = scoring_response.content[0].text.strip()
        score_match = re.search(r"\d+", score_text)
        score = int(score_match.group()) if score_match else 5  # Default to 5 if extraction fails

        if score < MIN_QUALITY_SCORE:
            print(f"⚠ Warning: AI-generated question scored low (score: {score}/10).")
            return {"valid": False, "reason": f"Low-quality response (score: {score}/10)."}

        return {"valid": True, "reason": "AI-generated follow-up question is high quality."}

    except Exception as e:
        print(f"Error in quality control check: {e}")
        return {"valid": False, "reason": "Quality control system failed."}


def generate_follow_up_question(session_id):
    """ Generates dynamic follow-up questions with quality control. """
    if session_id not in conversation_history or len(conversation_history[session_id]) == 0:
        raise ValueError("No conversation history found for session.")

    history = conversation_history[session_id]

    new_question = None
    is_valid = False
    attempts = 0
    while not is_valid and attempts < 5:  # Allow 5 attempts
        attempts += 1

        prompt = f"""
        You are a professional interviewer conducting a product management interview.
        Your goal is to generate a **natural and engaging** follow-up question while ensuring a dynamic, non-linear conversation.

        ### **Conversation Context**
        - **Original Interview Question:** "{history[0]['question']}"
        - **Last Question Asked:** "{history[-1]['question']}"
        - **Candidate’s Last Response:** "{history[-1]['answer']}"
        - **Previous Related Answers:** { [entry['answer'] for entry in history] }

        Now, based on the conversation flow, generate only one well-structured and engaging follow-up question at a time.
        """

        try:
            response = client.messages.create(
                model="claude-3-7-sonnet-20250219",
                max_tokens=150,
                system="You generate high-quality follow-up questions.",
                messages=[{"role": "user", "content": prompt}],
            )
            new_question = response.content[0].text.strip()

        except Exception as e:
            print(f"Error generating follow-up question: {e}")
            return "Sorry, an error occurred."

        check = quality_control_check(history[-1]["question"], history[-1]["answer"], new_question)
        is_valid = check["valid"]

    return new_question


#  Example Interview Session
session_id = "session_123"

if __name__ == "__main__":
    # Step 1: Generate Product Sense Question
    product_question = generate_product_sense_question()
    print(f"\n Product Sense Question: {product_question}")

    # Step 2: Get Candidate Answer
    candidate_answer = input("\n Enter candidate's answer: ")

    # Ensure session exists
    if session_id not in conversation_history:
        conversation_history[session_id] = []

    # Store the first question-answer pair
    conversation_history[session_id].append({"question": product_question, "answer": candidate_answer})

    # Step 3: Generate Follow-up Questions
    for i in range(3):  # Generate 3 follow-up questions
        follow_up_question = generate_follow_up_question(session_id)
        print(f"\n Follow-Up Question {i + 1}: {follow_up_question}")

        # Take user input as the candidate's answer
        candidate_answer = input("\n Enter candidate's answer: ")

        # Store question-answer pair in history
        conversation_history[session_id].append({"question": follow_up_question, "answer": candidate_answer})

    # Step 4: Generate Final Feedback After Entire Interview
    final_feedback = feedback_info_final(session_id)
    print("\n Final Interview Feedback:")
    print(final_feedback)



 📝 Product Sense Question: "Amazon's customer service department is looking to reduce average call resolution time without compromising customer satisfaction. How would you design a tool for customer service representatives that helps them resolve customer issues more efficiently while maintaining high-quality interactions?"

🗣️ Enter candidate's answer: I’d design Amazon SmartAssist, an AI-powered tool that provides real-time resolution suggestions, automated customer history summaries, and workflow automation to help service reps resolve issues faster. It would use NLP for smart search, sentiment analysis for tone guidance, and predictive assistance to recommend the best solutions. Routine tasks like refunds and status updates would be automated, while a live knowledge-sharing hub enables quick escalations. Success metrics would focus on reducing handling time, improving first-contact resolution, and maintaining high customer satisfaction (CSAT) scores.

 🔄 Follow-Up Question 1: Tel